In [4]:
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/")
def home():
    return {"message": "Hello from Jupyter Notebook!"}

Task exception was never retrieved
future: <Task finished name='Task-17' coro=<Server.serve() done, defined at C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nest_asyncio.py", line 31, in run
    return loop.run_until_complete(task)
  File "C:

INFO:     127.0.0.1:64709 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64709 - "GET /matches/cricket HTTP/1.1" 200 OK
INFO:     127.0.0.1:64718 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64718 - "GET /frontend/style.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:64718 - "GET /frontend/script.js HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22028]


In [16]:
import os
import requests
from fastapi import FastAPI, BackgroundTasks
import nest_asyncio
import uvicorn
from datetime import datetime
from pushbullet import Pushbullet
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles

# Initialize FastAPI
app = FastAPI()

# API Keys
import os

CRICKET_API_KEY = os.getenv("CRICKET_API_KEY", "b010864b-f0c8-414f-aebd-08f7996a1e69")
FOOTBALL_API_KEY = os.getenv("FOOTBALL_API_KEY", "c9751a3f6f4a4e7c9cd0721a71d6cd53")
PUSHBULLET_API_KEY = os.getenv("PUSHBULLET_API_KEY", "o.DV1U6GIbuhS3wrVhU82IywCrqmZyW2kD")

# Pushbullet instance
pb = Pushbullet(PUSHBULLET_API_KEY)

# API URLs
CRICKET_API_URL = f"https://api.cricapi.com/v1/currentMatches?apikey={CRICKET_API_KEY}"
FOOTBALL_API_URL = "https://api.football-data.org/v4/matches"
FOOTBALL_API_HEADERS = {"X-Auth-Token": FOOTBALL_API_KEY}

# Function to send notifications
def send_notification(title, message):
    pb.push_note(title, message)

# Fetch cricket matches
def get_cricket_matches():
    response = requests.get(CRICKET_API_URL)
    if response.status_code != 200:
        return {"error": "Failed to fetch Cricket API"}

    today = datetime.now().strftime('%Y-%m-%d')
    matches = response.json().get("data", [])
    upcoming_matches = [m for m in matches if m.get("date") >= today]

    if upcoming_matches:
        message = "\n".join([f"{m['name']} on {m['date']}" for m in upcoming_matches])
        send_notification("Upcoming Cricket Matches", message)
        return upcoming_matches
    return {"message": "No upcoming matches."}

# Fetch football matches
def get_football_matches():
    response = requests.get(FOOTBALL_API_URL, headers=FOOTBALL_API_HEADERS)
    if response.status_code != 200:
        return {"error": "Failed to fetch Football API"}

    matches = response.json().get("matches", [])
    upcoming_matches = [m for m in matches if m.get("status") == "SCHEDULED"]

    if upcoming_matches:
        message = "\n".join([f"{m['homeTeam']['name']} vs {m['awayTeam']['name']} on {m['utcDate']}" for m in upcoming_matches])
        send_notification("Upcoming Football Matches", message)
        return upcoming_matches
    return {"message": "No upcoming matches."}

# Schedule periodic task
def schedule_task():
    scheduler = BackgroundScheduler()
    scheduler.add_job(get_cricket_matches, IntervalTrigger(hours=1), id='cricket_task')
    scheduler.add_job(get_football_matches, IntervalTrigger(hours=1), id='football_task')
    scheduler.start()


# Startup event
@app.on_event("startup")
async def on_startup():
    schedule_task()

# Shutdown event
@app.on_event("shutdown")
async def on_shutdown():
    # You can add shutdown tasks here if necessary
    pass

# Serve homepage (index.html)
@app.get("/")
async def home():
    frontend_path = "C:/Users/KIIT/Desktop/sportsnoti/frontend/index.html"
    if os.path.exists(frontend_path):
        with open(frontend_path, "r") as file:
            return HTMLResponse(file.read())
    else:
        return {"error": "index.html not found"}

# Mount frontend static files (e.g., CSS, JS) for serving
app.mount("/frontend", StaticFiles(directory="C:/Users/KIIT/Desktop/sportsnoti/frontend"), name="frontend")

# API Endpoints
@app.get("/matches/cricket")
def cricket_endpoint(background_tasks: BackgroundTasks):
    background_tasks.add_task(get_cricket_matches)
    return {"message": "Checking for upcoming cricket matches."}

@app.get("/matches/football")
def football_endpoint(background_tasks: BackgroundTasks):
    background_tasks.add_task(get_football_matches)
    return {"message": "Checking for upcoming football matches."}

# Allow FastAPI in Jupyter
nest_asyncio.apply()

# Run server
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)



C:\Users\KIIT\AppData\Local\Temp\ipykernel_1684\1582784468.py:75: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
C:\Users\KIIT\AppData\Local\Temp\ipykernel_1684\1582784468.py:80: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("shutdown")
INFO:     Started server process [1684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53614 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53614 - "GET /style.css HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53615 - "GET /script.js HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53615 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53638 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53639 - "GET /frontend/script.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:53638 - "GET /frontend/style.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:53639 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53639 - "GET /matches/cricket HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1684]


Job "get_cricket_matches (trigger: interval[1:00:00], next run at: 2025-01-31 22:14:45 IST)" raised an exception
Traceback (most recent call last):
  File "C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1375, in getresp

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=10000)

In [17]:
!git init

Reinitialized existing Git repository in C:/Users/KIIT/Desktop/sportsnoti/.git/


In [18]:
!git remote remove origin
!git remote add origin https://github.com/Psnava123/sports-notification-app.git



In [19]:
!git add .

In [7]:
!git add .
!git status

On branch main
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   main.ipynb



In [9]:
!git commit -m "Uploading all project files"
!git push origin main




[main 00dd75d] Uploading all project files
 1 file changed, 13 insertions(+), 78 deletions(-)


To https://github.com/Psnava123/sports-notification-app.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/Psnava123/sports-notification-app.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [14]:
# Stage and commit all changes
!git add .
!git commit -m "Committing unstaged changes before pull"

# Pull the latest changes from GitHub with rebase
!git pull origin main --rebase

# Push your local changes to GitHub
!git push origin main



[main 26fe563] Committing unstaged changes before pull
 1 file changed, 70 insertions(+), 6 deletions(-)


From https://github.com/Psnava123/sports-notification-app
 * branch            main       -> FETCH_HEAD
error: cannot rebase: You have unstaged changes.
error: Please commit or stash them.
To https://github.com/Psnava123/sports-notification-app.git
 ! [rejected]        main -> main (non-fast-forward)
error: failed to push some refs to 'https://github.com/Psnava123/sports-notification-app.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [15]:
# 1. Stash any unstaged changes (if you don't want to commit them yet)
!git stash

# 2. Pull the latest changes from the remote repository (handling the rebase)
!git pull origin main --rebase

# 3. Apply the stashed changes back to your working directory
!git stash apply

# 4. Stage and commit the changes
!git add .
!git commit -m "Re-applied stashed changes"

# 5. Push the changes to the remote repository
!git push origin main


Saved working directory and index state WIP on main: 26fe563 Committing unstaged changes before pull


From https://github.com/Psnava123/sports-notification-app
 * branch            main       -> FETCH_HEAD
Rebasing (1/6)
Rebasing (2/6)
Rebasing (3/6)
Rebasing (4/6)
Rebasing (5/6)
Rebasing (6/6)

Successfully rebased and updated refs/heads/main.


On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   main.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main 6b4dbe9] Re-applied stashed changes
 1 file changed, 50 insertions(+), 3 deletions(-)


To https://github.com/Psnava123/sports-notification-app.git
   850b1de..6b4dbe9  main -> main
